<a href="https://colab.research.google.com/github/adampotton/Cognitive_AI_CW/blob/main/Q2A_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/neurogym/neurogym.git
%cd neurogym/
! pip install -e .

Cloning into 'neurogym'...
remote: Enumerating objects: 11100, done.
remote: Counting objects: 100% (1002/1002), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 11100 (delta 928), reused 896 (delta 896), pack-reused 10098 (from 1)
Receiving objects: 100% (11100/11100), 8.17 MiB | 14.94 MiB/s, done.
Resolving deltas: 100% (8333/8333), done.
/content/neurogym
Obtaining file:///content/neurogym
  Preparing metadata (setup.py) ... done
  Running setup.py develop for neurogym


In [10]:
import neurogym as ngym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_lstm_layers = 2, **kwargs):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_lstm_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_output, _ = self.lstm(x)
        out = self.fc(lstm_output)
        return out, lstm_output

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
config = {
    'dt': 100,                      # Set your timestep parameter
    'hidden_size': 32,             # Set the hidden size for your LSTM
    'batch_size': 16,               # Batch size for training
    'seq_len': 100,                 # Sequence length for input data
    'envid': 'ReadySetGo-v0',       # The task name
    'gain': 1,                      # Example: Custom gain
    'prod_margin': 50,              # Example: Custom production margin
}

env_kwargs = {
    'dt': config['dt'],
    'gain': config['gain'],        # Add gain parameter
    'prod_margin': config['prod_margin'],  # Add production margin parameter
}
config['env_kwargs'] = env_kwargs

dataset = ngym.Dataset(config['envid'], env_kwargs=config['env_kwargs'], batch_size=config['batch_size'], seq_len=config['seq_len'])
env = dataset.env

inputs, target = dataset()
inputs = torch.from_numpy(inputs).type(torch.float)

input_size = env.observation_space.shape[0]
output_size = env.action_space.n

print('Input has shape (SeqLen, Batch, Dim) =', inputs.shape)
print('Target has shape (SeqLen, Batch) =', target.shape)

iter_steps = 1000
report_freq = 100

# Create an instance of the Class RNNNet
net = LSTMNet(input_size, config['hidden_size'], output_size, num_lstm_layers = 1)
print(net)

def train_model(net, dataset, iter_steps, report_freq):

    # Use Adam optimizer
    optimizer = optim.AdamW(net.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    running_loss = 0
    running_acc = 0
    start_time = time.time()
    # Loop over training batches
    print('Training network...')
    for i in range(iter_steps):
        # Generate input and target, convert to pytorch tensor
        inputs, labels = dataset()
        inputs = torch.from_numpy(inputs).type(torch.float)
        labels = torch.from_numpy(labels.flatten()).type(torch.long)

        optimizer.zero_grad()   # zero the gradient buffers
        output, _ = net(inputs)
        output = output.view(-1, output_size)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()    # Does the update

        # Compute accuracy for the current batch
        batch_acc = (torch.argmax(output, dim=1) == labels).sum().item() / labels.shape[0]
        running_loss += loss.item()
        running_acc += batch_acc

        # Report every 100 steps
        if i % report_freq == report_freq - 1:
            running_loss /= report_freq
            running_acc /= report_freq  # average accuracy over the last 100 batches
            print('Step {}, Loss {:0.4f}, Accuracy {:0.4f}, Time {:0.1f}s'.format(
                i+1, running_loss, running_acc, time.time() - start_time))
            running_loss = 0
            running_acc = 0  # Reset for the next 100 steps
    return net

net = train_model(net, dataset, iter_steps, report_freq)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Input has shape (SeqLen, Batch, Dim) = torch.Size([100, 16, 3])
Target has shape (SeqLen, Batch) = (100, 16)
LSTMNet(
  (lstm): LSTM(3, 32)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)
Training network...
Step 100, Loss 0.1495, Accuracy 0.9706, Time 1.0s
Step 200, Loss 0.0975, Accuracy 0.9707, Time 2.0s
Step 300, Loss 0.0770, Accuracy 0.9706, Time 3.4s
Step 400, Loss 0.0624, Accuracy 0.9783, Time 4.9s
Step 500, Loss 0.0516, Accuracy 0.9869, Time 5.9s
Step 600, Loss 0.1094, Accuracy 0.9714, Time 7.0s
Step 700, Loss 0.0699, Accuracy 0.9742, Time 7.9s
Step 800, Loss 0.0663, Accuracy 0.9845, Time 9.1s
Step 900, Loss 0.0915, Accuracy 0.9742, Time 10.5s
Step 1000, Loss 0.0459, Accuracy 0.9879, Time 12.0s


In [47]:
env = dataset.env
env.reset(no_step=True)

# Reset environment
env = dataset.env
env.reset(no_step=True)

# Initialize variables for logging
perf = 0
activity_dict = {}  # recording activity
trial_infos = {}  # recording trial information

num_trial = 200
for i in range(num_trial):
    # Neurogym boiler plate
    # Sample a new trial
    trial_info = env.new_trial()
    # Observation and groud-truth of this trial
    ob, gt = env.ob, env.gt
    # Convert to numpy, add batch dimension to input
    inputs = torch.from_numpy(ob[:, np.newaxis, :]).type(torch.float)

    # Run the network for one trial
    # inputs (SeqLen, Batch, InputSize)
    # action_pred (SeqLen, Batch, OutputSize)
    action_pred, rnn_activity = net(inputs)

    # Compute performance
    # First convert back to numpy
    action_pred = action_pred.detach().numpy()[:, 0, :]
    # Read out final choice at last time step
    choice = np.argmax(action_pred[-1, :])
    # Compare to ground truth
    correct = choice == gt[-1]

    # Record activity, trial information, choice, correctness
    rnn_activity = rnn_activity[:, 0, :].detach().numpy()
    activity_dict[i] = rnn_activity
    trial_infos[i] = trial_info  # trial_info is a dictionary
    trial_infos[i].update({'correct': correct})
# Print information for sample trials
for i in range(10):
    print('Trial ', i, trial_infos[i])

print('Average performance', np.mean([val['correct'] for val in trial_infos.values()]))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Trial  0 {'measure': 1000.0, 'gain': 1, 'production': 1000.0, 'correct': True}
Trial  1 {'measure': 1000.0, 'gain': 1, 'production': 1000.0, 'correct': True}
Trial  2 {'measure': 800.0, 'gain': 1, 'production': 800.0, 'correct': True}
Trial  3 {'measure': 1300.0, 'gain': 1, 'production': 1300.0, 'correct': True}
Trial  4 {'measure': 900.0, 'gain': 1, 'production': 900.0, 'correct': True}
Trial  5 {'measure': 1200.0, 'gain': 1, 'production': 1200.0, 'correct': True}
Trial  6 {'measure': 800.0, 'gain': 1, 'production': 800.0, 'correct': True}
Trial  7 {'measure': 900.0, 'gain': 1, 'production': 900.0, 'correct': True}
Trial  8 {'measure': 1100.0, 'gain': 1, 'production': 1100.0, 'correct': True}
Trial  9 {'measure': 1400.0, 'gain': 1, 'production': 1400.0, 'correct': True}
Average performance 1.0
